In [47]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras import applications, optimizers

In [51]:
train_data = np.load('train_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])

test_data = np.load('test_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])

train = np.zeros([2000, 150, 150, 3])
for i in range(2000):
    train[i] = train_data[i]
train_labels = np.array([0] * 1000 + [1] * 1000)
    
test = np.zeros([800, 150, 150, 3])
for i in range(800):
    test[i] = test_data[i]
test_labels = np.array([0] * 400 + [1] * 400)

In [23]:
weight_path = 'bottleneck_fc_model.h5'
top_model_weight_path = 'first_try.h5'

In [24]:
batch_size = 16
epochs = 10

In [38]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

Model loaded.


In [39]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [40]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier in order to successfully do fine-tuning
top_model.load_weights(weight_path)

In [43]:
# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [44]:
# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [54]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [49]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [50]:
train_datagen.fit(train)
test_datagen.fit(test)

In [ ]:
model.fit_generator(train_datagen.flow(train, train_labels, batch_size=batch_size),
                    samples_per_epoch=train.shape[0] // batch_size, 
                    epochs=epochs, 
                    validation_data=test_datagen.flow(test, test_labels),
                    validation_steps=test.shape[0] // batch_size)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., validation_steps=50, steps_per_epoch=7)`
  """


Epoch 1/10
7/7 [==============================] - 758s 108s/step - loss: 0.6524 - acc: 0.8125 - val_loss: 0.6499 - val_acc: 0.8762
Epoch 2/10
6/7 [========================>.....] - ETA: 8s - loss: 0.5711 - acc: 0.8021 